In [1]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [2]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSz8Qs1gE_IYpzlkFkCXGcL_BqR8hZieWVi-rphN1gfrO3H4lDtVZs4kd0C3P8Y9lhsT1rhoB-Q_cP4/pubhtml'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# soup

In [4]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [5]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Sl_No',
 'Date',
 'Age Bracket',
 'Gender',
 'Patient_Status',
 'City',
 'District',
 'State',
 'Statecode',
 'Notes',
 'Nationality',
 'Source_1',
 'Source_2',
 'Source_3',
 'Patient_Number (Could be mapped later)',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [6]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [7]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Sl_No,Date,Age Bracket,Gender,Patient_Status,City,District,State,Statecode,Notes,...,Source_2,Source_3,Patient_Number (Could be mapped later),,,,,,,
0,59,13/03/2020,,,Deceased,,,Karnataka,KA,KA-P6,...,,,P74,,,,,,,
1,54,20/03/2020,,,Recovered,,,Karnataka,KA,KA-P5,...,,,,,,,,,,
2,55,24/03/2020,,,Recovered,,,Karnataka,KA,KA-P2,...,,,,,,,,,,
3,56,24/03/2020,,,Recovered,,,Karnataka,KA,KA-P4,...,,,,,,,,,,
4,60,26/03/2020,,,Deceased,,,Karnataka,KA,KA-P53,...,,,P690,,,,,,,


# Data cleaning and transformations

In [8]:
# shape of dataframe
# ==================

p_df.shape

(3932, 22)

In [9]:
# columns
# =======

p_df.columns

Index(['Sl_No', 'Date', 'Age Bracket', 'Gender', 'Patient_Status', 'City',
       'District', 'State', 'Statecode', 'Notes', 'Nationality', 'Source_1',
       'Source_2', 'Source_3', 'Patient_Number (Could be mapped later)', '',
       '', '', '', '', '', ''],
      dtype='object')

In [12]:
# selecting important columns only
# ================================

p_df = p_df.loc[:, :'Patient_Number (Could be mapped later)']

In [13]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Sl_No                                     0
Date                                      0
Age Bracket                               0
Gender                                    0
Patient_Status                            0
City                                      0
District                                  0
State                                     0
Statecode                                 0
Notes                                     0
Nationality                               0
Source_1                                  0
Source_2                                  0
Source_3                                  0
Patient_Number (Could be mapped later)    0
dtype: int64

In [14]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(3932, 15)
Sl_No 	 1
Date 	 63
Age Bracket 	 3585
Gender 	 3553
Patient_Status 	 63
City 	 3794
District 	 2821
State 	 63
Statecode 	 64
Notes 	 3134
Nationality 	 3916
Source_1 	 581
Source_2 	 3695
Source_3 	 3917
Patient_Number (Could be mapped later) 	 3872


In [15]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(3932, 15)


Sl_No                                        1
Date                                        63
Age Bracket                               3585
Gender                                    3553
Patient_Status                              63
City                                      3794
District                                  2821
State                                       63
Statecode                                   64
Notes                                     3134
Nationality                               3916
Source_1                                   581
Source_2                                  3695
Source_3                                  3917
Patient_Number (Could be mapped later)    3872
dtype: int64

In [16]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

KeyError: ['Detected State']

In [17]:
p_df.columns

Index(['Sl_No', 'Date', 'Age Bracket', 'Gender', 'Patient_Status', 'City',
       'District', 'State', 'Statecode', 'Notes', 'Nationality', 'Source_1',
       'Source_2', 'Source_3', 'Patient_Number (Could be mapped later)'],
      dtype='object')

In [18]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

,sl_no,date,age_bracket,gender,patient_status,city,district,state,statecode,notes,nationality,source_1,source_2,source_3,patient_number_(could_be_mapped_later)
2480,2474,17/04/2020,NaN,NaN,Recovered,NaN,NaN,Delhi,DL,NaN,NaN,https://twitter.com/ANI/status/125117866782225...,NaN,NaN,NaN
155,147,30/03/2020,NaN,NaN,Recovered,Pimpri-Chinchwad,Pune,Maharashtra,MH,4 Nagpur,NaN,https://twitter.com/rajeshtope11/status/124462...,NaN,NaN,NaN
1229,1221,11/04/2020,NaN,NaN,Recovered,NaN,NaN,Telangana,TG,NaN,NaN,https://t.me/indiacovid/2080,NaN,NaN,NaN
3525,3502,20/04/2020,NaN,NaN,Recovered,NaN,NaN,Tamil Nadu,TN,NaN,NaN,https://stopcorona.tn.gov.in/wp-content/upload...,NaN,NaN,NaN
2711,2696,18/04/2020,NaN,NaN,Recovered,NaN,NaN,Uttar Pradesh,UP,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

KeyError: 'patient_number'

In [20]:
p_df = p_df.loc[:, :'backup_notes']

KeyError: 'backup_notes'

# Saving Data

In [ ]:
p_df.sample(5)

In [ ]:
p_df.to_csv('patients_data.csv', index=False)